In [4]:
import openai

# call OpenAI API with model name and prompt
def call_openai_api(prompt, model_name='text-davinci-003', max_token=100, stop=None, n=1, temperature=0.1):
    openai.api_type = "azure"
    openai.api_version = "2022-12-01"
    
    openai.api_key = os.getenv('OPENAI_API_KEY') # replace this with your OpenAI API Key
    openai.api_base = os.getenv("OPENAI_API_BASE") # replace this with your OpenAI API Endpoint
    
    response = openai.Completion.create(
        engine=model_name,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_token,
        stop=stop,
        n=n,
    )
    return response


# Chain of Thought

Experiment results demonstrate Zero-shot-CoT using single prompt template, significantly outperform zero-shot LLM performance on diverse benchmark reasoning tasks.  Without any hand-crafted few-shot examples, e.g. increasing the accuracy on MultiArith from 17.7% to 78.7% and GSM8K from 10.4% to 40.7% with large InstructGPT model (text-davinci-002).

Source: [Large Language Models are Zero-Shot Reasoners](https://arxiv.org/abs/2205.11916)

In [6]:
# This prompt gets wrong answer

PROMPT_ZERO_SHOT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: The answer (arabic numerals) is
"""
response = call_openai_api(PROMPT_ZERO_SHOT, model_name='text-davinci-003', temperature=0, max_token=100)

print(response['choices'][0]['text'])

8.


In [7]:
# Still wrong answer with few-shot learning

PROMPT_FEW_SHOT = """Q: Roger has 5 tennis balss. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does Roger have now?
A: The answer is 11.

Q: A juggler can juggle 16 balls. Half of the balls are golf balls and half of the golf balls are blue. How many blue golf balls are there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT, model_name='text-davinci-003', temperature=0, max_token=100)

print(response['choices'][0]['text'])

The answer is 8.


In [8]:
# With CoT, the answer is correct

PROMPT_ZERO_SHOT_CoT = """Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A: Let’s think step by step.
"""
response = call_openai_api(PROMPT_ZERO_SHOT_CoT, model_name='text-davinci-003', temperature=0, max_token=100)

print(response['choices'][0]['text'])


First, the juggler can juggle 16 balls.

Second, half of the balls are golf balls. So, there are 8 golf balls.

Third, half of the golf balls are blue. So, there are 4 blue golf balls.


In [9]:

PROMPT_FEW_SHOT_CoT = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis
balls. Each can has 3 tennis balls. How many tennis balls does
he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls each is 6
tennis balls. 5 + 6 = 11. The answer is 11.
Q: A juggler can juggle 16 balls. Half of the balls are golf balls,
and half of the golf balls are blue. How many blue golf balls are
there?
A:
"""
response = call_openai_api(PROMPT_FEW_SHOT_CoT, model_name='text-davinci-003', temperature=0, max_token=100)

print(response['choices'][0]['text'])

Half of 16 balls is 8. Half of 8 golf balls is 4. Therefore, there are 4 blue golf balls.


# More research on CoT prompt engineering


# Program-aided Language Models 


In [10]:
PROMPT_FEW_SHOT_CoT = """Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
A: Roger started with 5 tennis balls. 2 cans of 3 tennis
balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The bakers at the Beverly Hills Bakery baked 200
loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery
store returned 6 unsold loaves. How many loaves of
bread did they have left?
"""
response = call_openai_api(PROMPT_FEW_SHOT_CoT, model_name='text-davinci-003', temperature=0, max_token=100)

print(response['choices'][0]['text'])

A: The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning. They sold 93 loaves in the morning and 39 loaves in the afternoon. 6 loaves were returned. 200 - (93 + 39 + 6) = 62. The answer is 62 loaves of bread left.


In [11]:
PROMPT_FEW_SHOT_PA = """Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?
A: Roger started with 5 tennis balls.
  tennis_balls = 5
2 cans of 3 tennis balls each is
  bought_balls = 2 * 3 
The answer is
answer = tennis_balls + bought_balls

Q: The bakers at the Beverly Hills Bakery baked 200
loaves of bread on Monday morning. They sold 93 loaves
in the morning and 39 loaves in the afternoon. A grocery
store returned 6 unsold loaves. How many loaves of bread
did they have left?
"""
response = call_openai_api(PROMPT_FEW_SHOT_PA, model_name='text-davinci-003', temperature=0, max_token=400)

print(response['choices'][0]['text'])

A: The bakers at the Beverly Hills Bakery baked 200 loaves of bread on Monday morning.
  baked_loaves = 200
They sold 93 loaves in the morning and 39 loaves in the afternoon.
  sold_loaves = 93 + 39
A grocery store returned 6 unsold loaves.
  returned_loaves = 6
The answer is
answer = baked_loaves - sold_loaves - returned_loaves


# Commonsense Reasoning

Paper: [Generated Knowledge Prompting for Commonsense Reasoning](https://arxiv.org/abs/2110.08387)



Provide knowledge, turn knowledge question into reasoning. In general, more knowledge, better result.

3 Contributing factors:

(i) the quality of knowledge, 

(ii) the quantity of knowledge where the performance improves with more knowledge statements, and 

(iii) the strategy for integrating knowledge during inference

In [12]:
PROMPT = """The player with the lowest score wins.
Is this true or false: Part of golf is trying to get a higher point total than others.
"""
response = call_openai_api(PROMPT, model_name='text-davinci-003', n=1, max_token=100)

print(response['choices'][0]['text'])

False


In [13]:
PROMPT = """A tripod is a kind of easel
How many legs does an easel have?
"""
response = call_openai_api(PROMPT, model_name='text-davinci-003', n=1, max_token=100)

print(response['choices'][0]['text'])


An easel typically has three legs.


## Check out follow 2 examples

In [14]:
# High confidence answer
PROMPT = """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. Each hole is played once in the round on a standard golf course. Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.

Explain and Answer: 
"""
response = call_openai_api(PROMPT, model_name='text-davinci-003', n=1, max_token=100)

print(response['choices'][0]['text'])

No, the objective of golf is not to get a higher point total than others. The objective of golf is to play a set of holes in the least number of strokes. The total number of strokes is used to determine the winner of the game, not the total number of points.


In [15]:
# Low confidence answer
PROMPT = """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: Golf is a precision club-and-ball sport in which competing players (or golfers) use many types of clubs to hit balls into a series of holes on a course using the fewest number of strokes. The goal is to complete the course with the lowest score, which is calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game.

Explain and Answer:
 
"""
response = call_openai_api(PROMPT, model_name='text-davinci-003', n=1, max_token=100)

print(response['choices'][0]['text'])

Yes, part of golf is trying to get a higher point total than others. The goal of the game is to complete the course with the lowest score, which is calculated by adding up the total number of strokes taken on each hole. The player with the lowest score wins the game, so players are competing to get the highest point total in order to win.
